In [ ]:
import socket
import datetime
import logging
import json
import os

PORT = int(input('Введите номер порта: '))
Log_Format = "%(levelname)s %(asctime)s: %(message)s"
with socket.socket() as sock:
    logging.basicConfig(filename = "logfile.log", filemode = "w", format = Log_Format, level = logging.DEBUG)
    logger = logging.getLogger()
    try:
        sock.bind(('', PORT))
        print('Используемый порт:', PORT)
    except OSError:
        logger.error('Порт занят')
        sock.bind(('', 0))
        PORT_new = sock.getsockname()[1]
        print('Используемый порт:', PORT_new)
    sock.listen(1)
    logger.info('Запуск сервера')
    while True:
        conn, addr = sock.accept()
        with conn:
            logger.info(f'Создано подключение: {addr}')
            address = conn.recv(1024).decode('utf-8')
            if os.stat('names.json').st_size == 0:
                names = []
                new = {}
                conn.send('None'.encode('utf-8'))
                name = conn.recv(1024).decode('utf-8')
                password = conn.recv(1024).decode('utf-8')
                new['address'] = address
                new['name'] = name
                new['password'] = password
                names.append(new)
                with open('names.json', 'w', encoding='utf-8') as f:
                    json.dump(names, f)
            else:
                with open('names.json', 'r', encoding='utf-8') as f:
                    names = json.load(f)
                    a = 0
                    for el in names:
                        if el['address'] == address:
                            name = el['name']
                            password = el['password']
                            conn.send(name.encode('utf-8'))
                            password_cl = conn.recv(1024).decode('utf-8')
                            a = 1
                            if password != password_cl:
                                logger.error('Неверный пароль')
                                conn.send('exit'.encode('utf-8'))
                            else:
                                conn.send('ok'.encode('utf-8'))
                if a == 0:
                    with open('names.json', 'r', encoding='utf-8') as f:
                        names = json.load(f)
                    new = {}
                    conn.send('None'.encode('utf-8'))
                    name = conn.recv(1024).decode('utf-8')
                    password = conn.recv(1024).decode('utf-8')
                    new['address'] = address
                    new['name'] = name
                    new['password'] = password
                    names.append(new)
                    with open('names.json', 'w', encoding='utf-8') as f:
                        json.dump(names, f, indent=1)
            while True:
                data = conn.recv(1024)
                if not data:
                    logger.info(f'Соединение с {addr} разорвано')
                    break
                data_decode = data.decode('utf-8')
                logger.info(f'Получены данные от клиента: {data_decode}')
                logger.info(f'Отправлены данные клиенту: {data_decode.upper()}')
                conn.send(bytes(data.decode('utf-8').upper(), 'utf-8'))
            logger.info(f'Закрываем соединение с {addr}')
            conn.close()
    logger.info('Отключение сервера')

Введите номер порта: 9091
Используемый порт: 60626
